In [ ]:
# default_exp leaderboard

# Leaderboard

> Visualising the public leaderboard.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
import typing
from fastcore.all import *
from ashrae import loading

In [3]:
pd.options.plotting.backend = "plotly"

In [4]:
%%time
csvs = loading.get_csvs()

2020-11-22 07:54:12.722 | INFO     | ashrae.loading:get_csvs:36 - Collected csv paths: {Path('../data/weather_test.csv'): 'weather_test', Path('../data/sample_submission.csv'): None, Path('../data/my_submission.csv'): None, Path('../data/test.csv'): 'test', Path('../data/train.csv'): 'train', Path('../data/ashrae-energy-prediction-publicleaderboard.csv'): 'public-leaderboard', Path('../data/weather_train.csv'): 'weather_train', Path('../data/building_metadata.csv'): 'building'}


CPU times: user 0 ns, sys: 2.59 ms, total: 2.59 ms
Wall time: 1.91 ms


In [5]:
leaderboard = pd.read_csv(csvs['public-leaderboard'], parse_dates=['SubmissionDate'])
leaderboard.head()

,TeamId,TeamName,SubmissionDate,Score
0,3794710,Marco Vasquez E,2019-10-16 05:08:39,4.077
1,3794722,sneaky goose,2019-12-09 11:28:50,1.071
2,3794730,Kostiantyn & Vladyslav,2019-11-12 20:45:47,1.071
3,3794765,Isamu & Matt,2019-12-19 21:50:14,0.938
4,3794772,Energy_leakage,2019-11-25 11:13:26,0.995


In [6]:
leaderboard['TeamId'].nunique(), len(leaderboard)

(3591, 3591)

In [7]:
#export
@typed
def get_leaderboard_distribution(df:pd.DataFrame):
    return (df.sort_values('Score')
              .groupby('Score')
              .size()
              .to_frame('Count')
              .reset_index()
              .assign(**{'Cumulative share (%)': lambda x: 100 * x['Count'].cumsum()/x['Count'].sum()}))

In [8]:
%%time
dis = get_leaderboard_distribution(leaderboard)
dis.head()

CPU times: user 9.56 ms, sys: 0 ns, total: 9.56 ms
Wall time: 65.1 ms


,Score,Count,Cumulative share (%)
0,0.931,1,0.027847
1,0.932,2,0.083542
2,0.933,1,0.111390
3,0.934,2,0.167084
4,0.935,1,0.194932


In [9]:
dis['Score'].describe(percentiles=[.05, .1, .25, .5, .75, .95])

count    940.000000
mean       1.822522
std        1.142008
min        0.931000
5%         0.977950
10%        1.029800
25%        1.176750
50%        1.444500
75%        2.135250
95%        3.725650
max       16.031000
Name: Score, dtype: float64

Public scores:

| Segment | Score |
| --- | --- |
| top 50% | 1.44 | 
| top 5% | 0.98 | 

With the best private leaderboard score being at 1.23 there is seems to be some overfitting / leakage in leading to those scores.

Line plot of the above

In [14]:
px.line(dis, x='Score', y='Cumulative share (%)', title='Cumulative distribution of public leaderboard scores')

ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil


Looking at the temporal trend of the scores to get an idea of jumps

In [ ]:
leaderboard.plot(kind='scatter', x='SubmissionDate', y='Score', title='Trend of the public score over time')

Finding:
- There are like 3 clusters around 1.243, 1.118 (from 2019-10-25 onwards) and 0.979 (from 2019-11-20 onwards) appearing over time

In [ ]:
from nbdev.export import *
notebook2script()